In [1]:
!pip install snowflake-snowpark-python[pandas]
!pip install snowflake-connector-python
!pip install snowflake-ml-python

     |████████████████████████████████| 1.2MB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 870kB 17.1MB/s eta 0:00:01
     |████████████████████████████████| 747kB 17.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 9.2MB/s  eta 0:00:01
     |████████████████████████████████| 2.5MB 18.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 10.1MB/s eta 0:00:01
     |████████████████████████████████| 153kB 21.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 9.5MB/s  eta 0:00:01
     |████████████████████████████████| 163kB 21.1MB/s eta 0:00:01
     |████████████████████████████████| 143kB 18.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 9.1MB/s  eta 0:00:01
     |████████████████████████████████| 71kB 10.2MB/s eta 0:00:01
     |████████████████████████████████| 112kB 20.6MB/s eta 0:00:01
     |████████████████████████████████| 512kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 450kB 12.2MB/s eta 0:00:01
  

In [ ]:
!pip install snowflake

     |████████████████████████████████| 450kB 1.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/9e/68/b3496e1d59e8c8b302df7fa5c4ce851a687b8b8ddb92d6f6254bfce77009/snowflake_snowpark_python-1.17.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
     |████████████████████████████████| 409kB 7.5MB/s eta 0:00:01
     |████████████████████████████████| 122kB 7.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/15/80/44286939ca215e88fa827b2aeb6fa3fd2b4a7af322485c7170d6f9fd96e0/cloudpickle-2.2.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7d/39/472f2554a0f1e825bd7c5afc11c817cd7a2f3657460f7159f691fbb37c51/PyYAML-6.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/95/c7/608cbdbaa37e6f0dfb22d19e1fa9365c113aacc10cfd6748a

  Using cached https://files.pythonhosted.org/packages/95/c7/608cbdbaa37e6f0dfb22d19e1fa9365c113aacc10cfd6748a148f91353d0/snowflake_connector_python-3.10.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/49/df/1fceb2f8900f8639e278b056416d49134fb8d84c5942ffaa01ad34782422/packaging-24.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c96257acb4fce61e7de14/certifi-2024.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/54/a7/2104f674a5a6845b04c8ff01659becc6b8978ca410b82b94287e0b1e018b/pyOpenSSL-24.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.wh

You should consider upgrading via the 'pip install --upgrade pip' command.


     |████████████████████████████████| 1.9MB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 174kB 60.7MB/s eta 0:00:01
     |████████████████████████████████| 41.0MB 54.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl
     |████████████████████████████████| 81kB 13.9MB/s eta 0:00:01
     |████████████████████████████████| 18.3MB 51.0MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 44.3MB/s eta 0:00:01
     |████████████████████████████████| 13.1MB 49.9MB/s eta 0:00:01
     |████████████████████████████████| 51kB 10.8MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 32.8MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 80kB/s  eta 0:00:01
     |████████████████████████████████| 133kB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 98.2MB 165kB/s eta 0:00:01
  Using cached https://files.p

     |████████████████████████████████| 348kB 1.6MB/s eta 0:00:01
     |████████████████████████████████| 8.3MB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 16.4MB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/7d/cd/d7460c9a869b16c3dd4e1e403cce337df165368c71d6af229a74699622ce/wheel-0.43.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/de/88/70c5767a0e43eb4451c2200f07d042a4bcd7639276003a9c54a68cfcc1f8/setuptools-70.0.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/32/46/9cb0e58b2deb7f82b84065f37f3bffeb12413f947f9388e4cac22c4621ce/sortedcontainers-2.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/79/fd/4525835d3e42e2db30169d42215ce74762f447fcbf01ed02f74f59bd74cb/cryptography-42.0.7-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c9/7f/0

  Using cached https://files.pythonhosted.org/packages/13/a3/a812df4e2dd5696d1f351d58b8fe16a405b234ad2886a0dab9183fb78109/pycparser-2.22-py3-none-any.whl
ERROR: snowflake-snowpark-python 1.17.0 has requirement cloudpickle!=2.1.0,!=2.2.0,<=2.2.1,>=1.6.0; python_version < "3.11", but you'll have cloudpickle 3.0.0 which is incompatible.
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 4.19.0 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.2.5 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.6.0; python_version >= "3.8", but you'll have matplotlib 3.9.0 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.9.0 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.7.3 which is incompatible.


You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from snowflake.snowpark import Session

In [ ]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [ ]:
session = get_session()
# session.close()
session

In [ ]:
REGISTRY_DATABASE_NAME = "test_database_prak"
REGISTRY_SCHEMA_NAME = "test_schema_prak"

In [2]:
from snowflake.ml.registry import model_registry

ModuleNotFoundError: No module named 'snowflake.ml'

In [3]:
!pip freeze | grep -i snow

snowflake-connector-python==3.10.1
snowflake-ml-python==1.5.0
snowflake-snowpark-python==1.17.0
